# RankMe: Getting Started Guide

This notebook demonstrates how to use the RankMe library to analyze and benchmark neural network models. We'll walk through various metrics including network analysis, feature learning assessment, and performance evaluation.

## Overview
RankMe provides comprehensive tools for:
- Network architecture analysis (parameters, size, FLOPs)
- Feature learning quality assessment using the RankMe score
- Standard ML metrics (accuracy, F1, IoU, MSE, etc.)
- Inference time benchmarking
- Complete model evaluation suites

## Model Setup

First, let's create a sample model to analyze. We'll use ResNet18 as our example throughout this tutorial.

In [1]:
import torchvision
import torch
model = torchvision.models.resnet18(weights=None)

## Individual Network Metrics

RankMe provides several network-level metrics to analyze your model's characteristics. Let's explore each one individually.

In [2]:
# Import the network metrics
from rankme.network import ParamCount, ModelSizeMB, Flops, PrecisionBits, InferenceTime

# Initialize the metrics
param_count = ParamCount()
model_size = ModelSizeMB()
flops = Flops()
precision_bits = PrecisionBits()
inference_time = InferenceTime()

# Calculate various network metrics for the ResNet18 model
print("=== Network Metrics for ResNet18 ===")

# 1. Parameter count
total_params = param_count(model)
trainable_params = param_count(model, include_non_trainable=False)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# 2. Model size in MB
size_params_only = model_size(model, include_buffers=False)
size_with_buffers = model_size(model, include_buffers=True)
print(f"Model size (parameters only): {size_params_only:.2f} MB")
print(f"Model size (with buffers): {size_with_buffers:.2f} MB")

# 3. FLOPs estimation (for typical ImageNet input 224x224)
input_size = (1, 3, 224, 224)  # Batch=1, Channels=3, Height=224, Width=224
estimated_flops = flops(model, input_size=input_size)
print(f"Estimated FLOPs: {estimated_flops:,.0f}")

# 4. Precision bits (average across all parameters)
avg_bits = precision_bits(model)
print(f"Average precision bits per parameter: {avg_bits:.1f}")

# Get dtype summary
dtype_summary = precision_bits.get_dtype_summary(model)
print("Parameter dtype summary:")
for dtype_name, count in dtype_summary.items():
    print(f"  {dtype_name}: {count:,} parameters")

# 5. Inference time measurement
cpu_time, gpu_time = inference_time(
    model, 
    input_size=(1, 3, 224, 224),  # ImageNet input size
    runs=50,  # Number of timing runs
    warmup=10  # Number of warmup runs
)
print(f"Average inference time (CPU): {cpu_time*1000:.2f} ms")
if not torch.isnan(torch.tensor(gpu_time)):
    print(f"Average inference time (GPU): {gpu_time*1000:.2f} ms")
else:
    print("GPU inference time: Not available (no CUDA device)")

=== Network Metrics for ResNet18 ===
Total parameters: 11,689,512
Trainable parameters: 11,689,512
Model size (parameters only): 44.59 MB
Model size (with buffers): 44.63 MB
Estimated FLOPs: 1,824,033,792
Average precision bits per parameter: 32.0
Parameter dtype summary:
  torch.float32: 11,689,512 parameters
Average inference time (CPU): 12.01 ms
GPU inference time: Not available (no CUDA device)


# Comprehensive Benchmark Suite

The `ModelBenchmark` class provides a unified interface to run all available metrics on your model. This is the recommended approach for complete model evaluation.

## What the Benchmark Includes:
- **Network Metrics**: Parameter count, model size, FLOPs, precision analysis, inference timing
- **Feature Learning**: RankMe score to assess representation quality
- **Classification Metrics**: Accuracy, F1-score, precision, recall, IoU
- **Regression Metrics**: MSE, MAE, RMSE, R² score

Let's set up our model and run the full benchmark:

In [3]:
import torchvision
model = torchvision.models.resnet18(weights=None)

In [4]:
import torch
import torch.nn as nn
from typing import Dict, Any, Tuple, Optional, Union
from collections import defaultdict
import time


# Import all available metrics
from rankme import RankMe, Accuracy, F1Score, IoU, Precision, Recall, MSE, MAE, R2Score, RMSE
from rankme.network import ParamCount, ModelSizeMB, Flops, PrecisionBits, InferenceTime
from rankme.suite import ModelBenchmark

# Example usage with the ResNet18 model
benchmark = ModelBenchmark()

# Generate some sample data for demonstration
batch_size, num_classes = 32, 10

# Sample embeddings (e.g., from model's feature extractor)
sample_embeddings = torch.randn(batch_size, 512)  # 512-dim features

# Sample classification data
y_true_class = torch.randint(0, num_classes, (batch_size,))
y_pred_class = torch.randint(0, num_classes, (batch_size,))

# Sample regression data  
y_true_reg = torch.randn(batch_size)
y_pred_reg = torch.randn(batch_size)

# Run comprehensive benchmark
print("Running comprehensive benchmark on ResNet18...")
results = benchmark.full_benchmark(
    model=model,
    embeddings=sample_embeddings,
    y_pred_class=y_pred_class,
    y_true_class=y_true_class, 
    y_pred_reg=y_pred_reg,
    y_true_reg=y_true_reg,
    num_classes=num_classes,
    input_size=(1, 3, 224, 224),
    runs=20,  # For inference timing
    warmup=5
)

# Pretty print all results
benchmark.print_results(results)

# You can also access individual results programmatically
print(f"\nQuick Summary:")
print(f"Parameters: {results['network']['total_parameters']:,}")
print(f"Model Size: {results['network']['size_total_mb']:.2f} MB") 
if 'feature_learning' in results:
    print(f"RankMe Score: {results['feature_learning']['rankme_score']:.4f}")
if 'classification' in results:
    print(f"Accuracy: {results['classification']['accuracy']:.4f}")
if 'regression' in results:
    print(f"R² Score: {results['regression']['r2_score']:.4f}")

Running comprehensive benchmark on ResNet18...
🚀 Starting Full Model Benchmark Suite
🔍 Computing Network Metrics...
📊 Computing Feature Learning Metrics...
🎯 Computing Classification Metrics...
📈 Computing Regression Metrics...

📊 MODEL BENCHMARK RESULTS

🏗️  Model: ResNet
📐 Input Size: (1, 3, 224, 224)

🔍 NETWORK METRICS
------------------------------
Total Parameters: 11,689,512
Trainable Parameters: 11,689,512
Model Size: 44.63 MB
FLOPs: 1,824,033,792
Avg Precision: 32.0 bits
CPU Inference: 11.71 ms

📊 FEATURE LEARNING METRICS
------------------------------
RankMe Score: 0.5543
RankMe (Centered): 0.5492

🎯 CLASSIFICATION METRICS
------------------------------
Accuracy: 0.1250
F1 Score: 0.1141
Precision: 0.1167
Recall: 0.1117
Iou: 0.0611

📈 REGRESSION METRICS
------------------------------
MSE: 1.6839
MAE: 1.0945
RMSE: 1.2976
R2_SCORE: -2.2361


Quick Summary:
Parameters: 11,689,512
Model Size: 44.63 MB
RankMe Score: 0.5543
Accuracy: 0.1250
R² Score: -2.2361
